In [ ]:
!nvidia-smi

Sun Sep  7 11:01:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="8EcvL4V9dJuQtjCY1ohJ")
project = rf.workspace("tutorial-extpj").project("brain-tumor-segmentation-jteuo")
version = project.version(1)
dataset = version.download("yolov11")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Brain-Tumor-Segmentation-1 in yolov11:: 100%|██████████| 6480/6480 [00:00<00:00, 7045.42it/s]


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.1 MB/s eta 0:00:00


In [ ]:
import os
HOME = os.getcwd()
print(HOME)


/content


In [ ]:
from ultralytics import YOLO



In [ ]:
%%writefile segment_efficient.py
import torch
import torch.nn as nn
from torch import Tensor

# Depthwise separable conv
class DWConv(nn.Module):
    def __init__(self, c1, c2, k=3, s=1, p=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(c1, c1, k, s, p, groups=c1, bias=False),
            nn.BatchNorm2d(c1),
            nn.ReLU(),
            nn.Conv2d(c1, c2, 1, 1, 0, bias=False),
            nn.BatchNorm2d(c2),
            nn.ReLU()
        )
    def forward(self, x):
        return self.conv(x)

# Placeholder for Detect base class
class Detect(nn.Module):
    def __init__(self, nc=80, ch=()):
        super().__init__()
        self.nc = nc
        self.nl = len(ch)
    def forward(self, x):
        return x

# Placeholder for Proto module
class Proto(nn.Module):
    def __init__(self, c_in, npr, nm):
        super().__init__()
        self.conv = nn.Conv2d(c_in, npr, 1)
        self.nm = nm
    def forward(self, x):
        return self.conv(x)

# Efficient Segment Head
class SegmentEfficient(Detect):
    def __init__(self, nc=80, nm=32, npr=256, ch=()):
        super().__init__(nc, ch)
        self.nm = nm
        self.npr = npr
        self.proto = Proto(ch[0], self.npr, self.nm)
        c4 = max(ch[0] // 8, self.nm)
        self.cv4 = nn.ModuleList(
            nn.Sequential(DWConv(x, c4), nn.Conv2d(c4, self.nm, 1)) for x in ch
        )
    def forward(self, x: list[Tensor]) -> tuple | list[Tensor]:
        p = self.proto(x[0])
        bs = p.shape[0]
        mc = torch.cat([self.cv4[i](x[i]).view(bs, self.nm, -1) for i in range(self.nl)], 2)
        x_out = Detect.forward(self, x)
        if self.training:
            return x_out, mc, p
        return (torch.cat([x_out, mc], 1), p) if getattr(self, "export", False) else (torch.cat([x_out[0], mc], 1), (x_out[1], mc, p))


Writing segment_efficient.py


In [ ]:
from segment_efficient import SegmentEfficient

# Register for YOLO YAML parser
globals()['SegmentEfficient'] = SegmentEfficient

In [ ]:
%%writefile segment_efficient.py
import torch
import torch.nn as nn
from torch import Tensor

class DWConv(nn.Module):
    def __init__(self, c1, c2, k=3, s=1, p=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(c1, c1, k, s, p, groups=c1, bias=False),
            nn.BatchNorm2d(c1),
            nn.ReLU(),
            nn.Conv2d(c1, c2, 1, 1, 0, bias=False),
            nn.BatchNorm2d(c2),
            nn.ReLU()
        )
    def forward(self, x):
        return self.conv(x)

class Detect(nn.Module):
    def __init__(self, nc=80, ch=()):
        super().__init__()
        self.nc = nc
        self.nl = len(ch)
    def forward(self, x):
        return x

class Proto(nn.Module):
    def __init__(self, c_in, npr, nm):
        super().__init__()
        self.conv = nn.Conv2d(c_in, npr, 1)
        self.nm = nm
    def forward(self, x):
        return self.conv(x)

class SegmentEfficient(Detect):
    def __init__(self, nc=80, nm=32, npr=256, ch=()):
        super().__init__(nc, ch)
        self.nm = nm
        self.npr = npr
        self.proto = Proto(ch[0], self.npr, self.nm)
        c4 = max(ch[0] // 8, self.nm)
        self.cv4 = nn.ModuleList(
            nn.Sequential(DWConv(x, c4), nn.Conv2d(c4, self.nm, 1)) for x in ch
        )
    def forward(self, x: list[Tensor]) -> tuple | list[Tensor]:
        p = self.proto(x[0])
        bs = p.shape[0]
        mc = torch.cat([self.cv4[i](x[i]).view(bs, self.nm, -1) for i in range(self.nl)], 2)
        x_out = Detect.forward(self, x)
        if self.training:
            return x_out, mc, p
        return (torch.cat([x_out, mc], 1), p) if getattr(self, "export", False) else (torch.cat([x_out[0], mc], 1), (x_out[1], mc, p))


Overwriting segment_efficient.py


In [ ]:
from ultralytics import YOLO
from segment_efficient import SegmentEfficient  # import your new head

# Load YOLO with YAML (placeholder Segment head is fine)
model = YOLO("/content/seg_yolo11n.yaml")

# Replace the head with your new SegmentEfficient
nc = 1                   # number of classes
nm = 32                  # number of masks
npr = 256                # number of protos
ch = (256, 512, 1024)    # backbone channels

model.model.head = SegmentEfficient(nc=nc, nm=nm, npr=npr, ch=ch)


In [ ]:
print(model.model)


SegmentationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      

In [ ]:
model = YOLO("/content/seg_yolo11n.yaml")

KeyError: 'SegmentEfficient'

In [ ]:
# Step 3: Verify replacement
print(model.model.head)          # Should display SegmentEfficient architecture
print(type(model.model.head))    # Should output: <class 'segment_efficient.SegmentEfficient'>

# Optional: check modules inside the head
for name, module in model.model.head.named_modules():
    print(name, module)


AttributeError: 'SegmentationModel' object has no attribute 'head'

In [ ]:
model.train(data="/content/Brain-Tumor-Segmentation-1/data.yaml", epochs=3 , imgsz=512, batch=16)

Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Brain-Tumor-Segmentation-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/seg_yolo11n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x780dddd535c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
#model = YOLO("/content/jm_yolo11_fracture.yaml")   # upload yaml data path run
model.train(data="/content/Brain-Tumor-Segmentation-1/data.yaml", epochs=3 , imgsz=512, batch=16)

Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Brain-Tumor-Segmentation-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/seg_yolo11n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x780e93f67fb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
results = model.predict(source="/content/Brain-Tumor-Segmentation-1/test/images", save=True)


image 1/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0022_jpg.rf.a1ba56ae56c15bc4e33eae59954487fd.jpg: 512x512 2 meningiomas, 15.2ms
image 2/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0032_jpg.rf.9a8f0b8aa368e38f21b7b3345f553d43.jpg: 512x512 1 meningioma, 13.0ms
image 3/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0046_jpg.rf.e641c817cec9d9029adaa4bdd51d5ffc.jpg: 512x512 (no detections), 13.0ms
image 4/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0052_jpg.rf.87e85ebc4361c148a4ca8ed0fa59f011.jpg: 512x512 (no detections), 12.9ms
image 5/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0058_jpg.rf.b0647bde5294b0580bebf39517e3d0f3.jpg: 512x512 1 meningioma, 12.9ms
image 6/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0061_jpg.rf.1861f09f43d770af9dc798693fd4b93e.jpg: 512x512 1 meningioma, 13.0ms
image 7/324 /content/Brain-Tumor-Segmentation-1/test/images/Tr-me_0062_jpg.rf.f481b888102a6c630c6a3cac16aa5bc1.jpg: 512x51

In [ ]:
import cv2
import numpy as np
import glob
import os
import pandas as pd

# Path to your YOLO output directory
mask_dir = "/content/masks"

# Find all mask files
mask_paths = glob.glob(os.path.join(mask_dir, "**/*_mask*.png"), recursive=True)
print(f"Found {len(mask_paths)} mask files")

results = []

for mask_path in mask_paths:
    # Load mask
    mask = cv2.imread(mask_path, 0)  # grayscale
    if mask is None:
        continue

    # Binary mask (1 = tumor, 0 = background)
    mask_bin = (mask > 0).astype(np.uint8)

    # Calculate tumor area in pixels
    tumor_pixels = np.sum(mask_bin)

    # Total image area
    h, w = mask.shape
    total_pixels = h * w
    tumor_percent = (tumor_pixels / total_pixels) * 100

    # Save result
    img_name = os.path.basename(mask_path).replace("_mask0.png", "")
    results.append([img_name, tumor_pixels, total_pixels, tumor_percent])

# Convert to DataFrame
df = pd.DataFrame(results, columns=["Image", "Tumor Pixels", "Total Pixels", "Tumor %"])

# Save to CSV
output_csv = "tumor_area_results.csv"
df.to_csv(output_csv, index=False)

print(f"✅ Calculated tumor area for {len(results)} images.")
print(f"Results saved to: {output_csv}")


Found 486 mask files
✅ Calculated tumor area for 486 images.
Results saved to: tumor_area_results.csv


In [ ]:
row_spacing = 0.5  # mm
col_spacing = 0.5  # mm
pixel_area_mm2 = row_spacing * col_spacing

df["Tumor Area (mm²)"] = df["Tumor Pixels"] * pixel_area_mm2
df.to_csv("tumor_area_results_mm2.csv", index=False)


In [ ]:
import pandas as pd

# Load your CSV
csv_file = "/content/tumor_area_results_mm2.csv"  # replace with your CSV file name
df = pd.read_csv(csv_file)

# Function to classify by tumor area in mm²
def tumor_category_mm2(area_mm2):
    if area_mm2 < 100:
        return "Small"
    elif area_mm2 < 500:
        return "Medium"
    else:
        return "Large"

# Add Category column
df["Category"] = df["Tumor Area (mm²)"].apply(tumor_category_mm2)

# Save updated CSV
output_csv = "tumor_area_classified_mm2.csv"
df.to_csv(output_csv, index=False)

print(f"✅ Tumors classified by mm². Results saved to: {output_csv}")


✅ Tumors classified by mm². Results saved to: tumor_area_classified_mm2.csv


In [ ]:
import cv2
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
import glob
import os
import pandas as pd

# Path to YOLO mask folder
mask_dir = "/content/masks"  # change if needed
mask_paths = glob.glob(os.path.join(mask_dir, "**/*_mask*.png"), recursive=True)

results = []

for mask_path in mask_paths:
    # Load mask
    mask = cv2.imread(mask_path, 0)
    if mask is None:
        continue

    # Convert to binary
    mask_bin = (mask > 0).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert contours to Shapely polygons
    polygons = []
    for cnt in contours:
        if len(cnt) >= 3:  # need at least 3 points for a polygon
            poly_points = [(point[0][0], point[0][1]) for point in cnt]
            polygons.append(Polygon(poly_points))

    # Merge multiple polygons into one MultiPolygon
    if len(polygons) == 0:
        area_pixels = 0
    elif len(polygons) == 1:
        area_pixels = polygons[0].area
    else:
        multi = MultiPolygon(polygons)
        area_pixels = multi.area

    # Optional: Convert to mm² if pixel spacing is known
    row_spacing = 0.5  # mm per pixel (replace with your value)
    col_spacing = 0.5  # mm per pixel
    area_mm2 = area_pixels * row_spacing * col_spacing

    # Total image pixels
    h, w = mask.shape
    tumor_percent = (area_pixels / (h * w)) * 100

    # Save results
    img_name = os.path.basename(mask_path).replace("_mask0.png", "")
    results.append([img_name, area_pixels, area_mm2, tumor_percent])

# Save results to CSV
df = pd.DataFrame(results, columns=["Image", "Tumor Area (pixels)", "Tumor Area (mm²)", "Tumor %"])
output_csv = "tumor_area_polygon.csv"
df.to_csv(output_csv, index=False)

print(f"✅ Calculated tumor area for {len(results)} masks using polygon approximation.")
print(f"Results saved to: {output_csv}")


✅ Calculated tumor area for 486 masks using polygon approximation.
Results saved to: tumor_area_polygon.csv


In [ ]:
import pandas as pd

# Load your CSV
csv_file = "/content/tumor_area_polygon.csv"  # replace with your actual CSV file name
df = pd.read_csv(csv_file)

# Function to classify tumor based on mm²
def classify_tumor(area_mm2):
    if area_mm2 < 100:
        return "Small"
    elif area_mm2 < 500:
        return "Medium"
    else:
        return "Large"

# Apply classification
df["Category"] = df["Tumor Area (mm²)"].apply(classify_tumor)

# Save updated CSV
output_csv = "tumor_area_classified.csv"
df.to_csv(output_csv, index=False)

print(f"✅ Tumors classified. Results saved to: {output_csv}")


✅ Tumors classified. Results saved to: tumor_area_classified.csv
